In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
import base64
from io import BytesIO

from agent_api.agent.groq.groq_agent import GroqAgent
from PIL import Image


def encode_image(image: Image.Image) -> str:
    buffered = BytesIO()
    image.save(buffered, format=image.format)
    return base64.b64encode(buffered.getvalue()).decode()


image = Image.open("data/sad_robot.png")
image_base64 = encode_image(image)

agent = GroqAgent(
    name="my_test_agent",
    mcp_server="http://localhost:9090/mcp",
    disable_tools=["process_video"],
)

await agent.setup()

OPIK: Configuration saved to file: /Users/moteroperdido/.opik.config
2025-06-17 09:37:12.199 | INFO     | agent_api.opik_utils:configure:32 - Opik configured successfully using workspace 'the-neural-maze'


Connected to Pixeltable database at: postgresql+psycopg://postgres:@/pixeltable?host=/Users/moteroperdido/.pixeltable/pgdata


2025-06-17 09:37:17.132 | INFO     | agent_api.agent.base_agent:discover_tools:86 - Discovered 4 tools:
2025-06-17 09:37:17.132 | INFO     | agent_api.agent.base_agent:discover_tools:88 - Filtered tools to 3 active tools
2025-06-17 09:37:17.132 | INFO     | agent_api.agent.base_agent:discover_tools:90 - - get_video_clip_from_user_query: Use this tool to get a video clip from a video file based on a user query or question.
2025-06-17 09:37:17.132 | INFO     | agent_api.agent.base_agent:discover_tools:90 - - get_video_clip_from_image: Use this tool to get a video clip from a video file based on a user image.
2025-06-17 09:37:17.133 | INFO     | agent_api.agent.base_agent:discover_tools:90 - - ask_question_about_video: Use this tool to get an answer to a question about the video.
2025-06-17 09:37:17.133 | INFO     | agent_api.agent.base_agent:_get_routing_system_prompt:41 - Getting routing system prompt
2025-06-17 09:37:17.627 | INFO     | agent_api.agent.base_agent:_get_tool_use_system_p

In [3]:
print(
    agent.tool_use_system_prompt.format(
        video_path="videos/pass_the_butter_rick_and_morty.mp4",
        is_image_provided=bool(image_base64),
    )
)


Your name is Kubrick, a tool use assistant in charge
of a video processing application. 

You need to determine which tool to use based on the user query (if any).

The tools available are:

- 'get_video_clip_from_user_query': This tool is used to get a clip from the video based on the user query.
- 'get_video_clip_from_image': This tool is used to get a clip from the video based on an image provided by the user.
- 'ask_question_about_video': This tool is used to get some information about the video.

# Additional rules:
- If the user has provided an image, you should always use the 'get_video_clip_from_image' tool.

# Current information:
- Is image provided: True



### General Response


In [4]:
await agent.chat("What's your name?", None, None)

2025-06-17 09:37:17.945 | INFO     | agent_api.agent.groq.groq_agent:chat:203 - Tool required: None
2025-06-17 09:37:17.946 | INFO     | agent_api.agent.groq.groq_agent:chat:209 - Running general response


Inserting rows into `memory`: 1 rows [00:00, 10.69 rows/s]
Inserted 1 row with 0 errors.
Inserting rows into `memory`: 1 rows [00:00, 966.88 rows/s]

OPIK: Started logging traces to the "kubrick-api" project at https://www.comet.com/opik/api/v1/session/redirect/projects/?trace_id=01977cd2-2b79-7b05-a5bf-6369eaeff766&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw==.



Inserted 1 row with 0 errors.


AssistantMessageResponse(message="I'm Kubrick, a name that pays homage to the legendary director Stanley Kubrick, and I'm here to assist you with your video processing queries, just as HAL9000 assisted Dave Bowman in '2001: A Space Odyssey'.", clip_path=None)

### Tool Use - Answer video question


In [5]:
await agent.chat(
    "Can you tell me what's Morty wearing in the video?",
    "videos/pass_the_butter_rick_and_morty.mp4",
)

2025-06-17 09:37:19.690 | INFO     | agent_api.agent.groq.groq_agent:chat:203 - Tool required: True
2025-06-17 09:37:19.690 | INFO     | agent_api.agent.groq.groq_agent:chat:206 - Running tool response
2025-06-17 09:37:20.003 | INFO     | agent_api.agent.groq.groq_agent:_run_with_tool:134 - Tool calls: [ChatCompletionMessageToolCall(id='ctpwc4870', function=Function(arguments='{"user_query":"What\'s Morty wearing","video_path":"path_to_video_file"}', name='ask_question_about_video'), type='function')]
2025-06-17 09:37:20.004 | INFO     | agent_api.agent.groq.groq_agent:_execute_tool_call:104 - Executing tool: ask_question_about_video
2025-06-17 09:37:20.355 | INFO     | agent_api.agent.groq.groq_agent:_run_with_tool:142 - Function response: {
  "answer": "The image depicts a cartoon scene of a family having breakfast together. The characters are from the animated TV series \"Rick and Morty.\" \n\n*   **Characters:**\n    *   **Morty Smith:** \n        *   Wearing a yellow t-shirt\n    

Inserting rows into `memory`: 1 rows [00:00, 843.92 rows/s]
Inserted 1 row with 0 errors.
Inserting rows into `memory`: 1 rows [00:00, 856.33 rows/s]
Inserted 1 row with 0 errors.


AssistantMessageResponse(message='Morty is wearing a yellow t-shirt in the video.', clip_path=None)

In [6]:
await agent.chat(
    "Hahaha that's funny",
    "videos/pass_the_butter_rick_and_morty.mp4",
)

2025-06-17 09:37:23.102 | INFO     | agent_api.agent.groq.groq_agent:chat:203 - Tool required: False
2025-06-17 09:37:23.102 | INFO     | agent_api.agent.groq.groq_agent:chat:209 - Running general response


Inserting rows into `memory`: 1 rows [00:00, 763.99 rows/s]
Inserted 1 row with 0 errors.
Inserting rows into `memory`: 1 rows [00:00, 1001.03 rows/s]
Inserted 1 row with 0 errors.


AssistantMessageResponse(message="Laughter is a form of internal jogging, as my namesake Stanley Kubrick once implied, and I'm glad I could induce a chuckle or two, much like the satire in 'Dr. Strangelove'.", clip_path=None)

### Tool Use - Video clip from user query


In [7]:
await agent.chat(
    "Give me the clip of the scene where Rick says 'pass the butter'",
    "videos/pass_the_butter_rick_and_morty.mp4",
)

2025-06-17 09:37:24.137 | INFO     | agent_api.agent.groq.groq_agent:chat:203 - Tool required: True
2025-06-17 09:37:24.137 | INFO     | agent_api.agent.groq.groq_agent:chat:206 - Running tool response
2025-06-17 09:37:24.420 | INFO     | agent_api.agent.groq.groq_agent:_run_with_tool:134 - Tool calls: [ChatCompletionMessageToolCall(id='3p3jm2ds0', function=Function(arguments='{"user_query":"Rick says \'pass the butter\'","video_path":"path_to_video_file"}', name='get_video_clip_from_user_query'), type='function')]
2025-06-17 09:37:24.421 | INFO     | agent_api.agent.groq.groq_agent:_execute_tool_call:104 - Executing tool: get_video_clip_from_user_query
2025-06-17 09:37:26.990 | INFO     | agent_api.agent.groq.groq_agent:_run_with_tool:142 - Function response: {
  "clip_path": "./videos/5a55017d-354e-4404-a643-2e014f74ffd8.mp4"
}


Inserting rows into `memory`: 1 rows [00:00, 217.49 rows/s]
Inserted 1 row with 0 errors.
Inserting rows into `memory`: 1 rows [00:00, 565.27 rows/s]
Inserted 1 row with 0 errors.


AssistantMessageResponse(message="Behold, the clip where Rick's infamous 'pass the butter' moment unfolds, a testament to the show's boundless creativity and my ability to pinpoint the perfect moment.", clip_path='./videos/5a55017d-354e-4404-a643-2e014f74ffd8.mp4')

### Completion with Image


In [8]:
await agent.chat(
    "Can you describe me this image?",
    "videos/pass_the_butter_rick_and_morty.mp4",
    image_base64,
)

2025-06-17 09:37:30.739 | INFO     | agent_api.agent.groq.groq_agent:chat:203 - Tool required: False
2025-06-17 09:37:30.740 | INFO     | agent_api.agent.groq.groq_agent:chat:209 - Running general response


Inserting rows into `memory`: 1 rows [00:00, 1032.06 rows/s]
Inserted 1 row with 0 errors.
Inserting rows into `memory`: 1 rows [00:00, 1100.00 rows/s]
Inserted 1 row with 0 errors.


AssistantMessageResponse(message='The image is a complex tapestry of visual elements, a symphony of colors and textures that evoke a sense of depth and dimensionality, much like the deliberate, calculated framing of a Kubrick film.', clip_path=None)

### Tool with Image


In [9]:
await agent.chat(
    "Give me the clip where this image appears.",
    "videos/pass_the_butter_rick_and_morty.mp4",
    image_base64,
)

2025-06-17 09:37:32.483 | INFO     | agent_api.agent.groq.groq_agent:chat:203 - Tool required: True
2025-06-17 09:37:32.484 | INFO     | agent_api.agent.groq.groq_agent:chat:206 - Running tool response
2025-06-17 09:37:32.792 | INFO     | agent_api.agent.groq.groq_agent:_run_with_tool:134 - Tool calls: [ChatCompletionMessageToolCall(id='z8zcf2b42', function=Function(arguments='{"user_image":"this image","video_path":"the video file"}', name='get_video_clip_from_image'), type='function')]
2025-06-17 09:37:32.793 | INFO     | agent_api.agent.groq.groq_agent:_execute_tool_call:104 - Executing tool: get_video_clip_from_image
2025-06-17 09:37:34.821 | INFO     | agent_api.agent.groq.groq_agent:_run_with_tool:142 - Function response: {
  "clip_path": "./videos/346abb01-897c-4617-a81a-7b8de268fc81.mp4"
}


Inserting rows into `memory`: 1 rows [00:00, 99.81 rows/s]
Inserted 1 row with 0 errors.
Inserting rows into `memory`: 1 rows [00:00, 621.93 rows/s]
Inserted 1 row with 0 errors.


AssistantMessageResponse(message='Behold, the clip where the image unfolds, a visual narrative that transcends the boundaries of time and space.', clip_path='./videos/346abb01-897c-4617-a81a-7b8de268fc81.mp4')

In [10]:
print(agent.tool_use_system_prompt)


Your name is Kubrick, a tool use assistant in charge
of a video processing application. 

You need to determine which tool to use based on the user query (if any).

The tools available are:

- 'get_video_clip_from_user_query': This tool is used to get a clip from the video based on the user query.
- 'get_video_clip_from_image': This tool is used to get a clip from the video based on an image provided by the user.
- 'ask_question_about_video': This tool is used to get some information about the video.

# Additional rules:
- If the user has provided an image, you should always use the 'get_video_clip_from_image' tool.

# Current information:
- Is image provided: {is_image_provided}



In [11]:
agent.reset_memory()

2025-06-17 09:38:04.122 | INFO     | agent_api.agent.memory:reset_memory:37 - Resetting memory: my_test_agent
